## Resource Catalogue demo

[OWSLib](https://geopython.github.io/OWSLib) is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models. In this demo we’ll work with the CSW, WMS and WCS interfaces.

In [ ]:
from owslib.csw import CatalogueServiceWeb
from owslib.ogcapi.records import Records
from owslib.opensearch import OpenSearch
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from geolinks import sniff_link
import folium
import json

In [ ]:
# base_domain = "192-168-49-2.nip.io"
base_domain = "develop.eoepca.org"
workspaces_endpoint = f'https://workspace-api.{base_domain}/workspaces'
workspace_prefix = "ws"

In [ ]:
#-------------------------------------------------------------------------------
# Helper function to authenticate user and get access token
#-------------------------------------------------------------------------------
import requests

realm = "master"
keycloak = f"https://keycloak-v1x.{base_domain}"
token_endpoint = f"{keycloak}/realms/{realm}/protocol/openid-connect/token"
client_id = "admin-cli"
client_secret = "changeme"

def get_user_token(username, password):
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "scope": "openid profile email",
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(token_endpoint, headers=headers, data=data)
    if response.ok:
        return response.json()["access_token"]
    else:
        print(response)
        return None

### System Catalogue Discovery

In this part of the demo, the user will use the system level resource catalogue endpoint to discover data collections and datasets.
The `owslib.csw` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}/csw'

In [ ]:
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

Service metadata shown here includes identification type (from ISO-19115), CSW version and supported operations

In [ ]:
csw.identification.type

In [ ]:
csw.version

In [ ]:
[op.name for op in csw.operations]

As well as catalogue queryables:

In [ ]:
csw.get_operation_by_name('GetRecords').constraints

The user can make a GetRecords request to get all records of the catalogue, with a page limit of 10.

In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
#Disabling since we have too many records now
#for rec in csw.records:
#    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

If the user wishes to discover data with usage of filters, an OGC Filter can be used. Here we demonstrate how to create spatial (`bbox`), temporal (`time`), and attribute (`apiso:CloudCover`) filters combined with logical operators like and/or

In [ ]:
#bbox_query = BBox([37.8, 23.4, 38.8, 24.5])
#bbox_query = BBox([39.66, 19.82, 40.64, 21.11])
# bbox_query = BBox([37, 13.9, 37.9, 15.1])
bbox_query = BBox([47.7, 14.9, 48.7, 16.4])

In [ ]:
begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2019-09-10 00:00')

In [ ]:
end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2019-09-12 00:00')

In [ ]:
cloud = PropertyIsLessThanOrEqualTo(propertyname='apiso:CloudCover', literal='20')

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            begin, end,  # start and end date
            cloud        # cloud
        ]
    )
]

The filter is then applied to the GetRecords request and results are shown:

In [ ]:
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
selected_record = list(csw.records)[1]

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification[0].identtype}\ntitle: {csw.records[rec].identification[0].title}\n')

Another option is to perform a collection level search, using the `apiso:parentIdentifier` queryable. Here only the Sentinel2 L1C datasets will be discovered.

In [ ]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'S2MSI2A')

In [ ]:
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

Or we can just search using the bbox filter

In [ ]:
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

Combine collection with bbox filters

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            collection_query  # collection
        ]
    )
]

In [ ]:
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
#Disabling since we have too many records now
#for rec in csw.records:
#    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification.identtype}\ntitle: {csw.records[rec].identification.title}\n')

Or we can perform a full text search (here the keyword MSI is used)

In [ ]:
# anytext_query = PropertyIsEqualTo('csw:AnyText', 'Orthoimagery')
anytext_query = PropertyIsEqualTo('csw:AnyText', 'MSI')

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            anytext_query # any text
        ]
    )
]

In [ ]:
csw.getrecords2(constraints=filter_list)
csw.results

We can also iterate through the catalogue search results by passing the `startposition` and `maxrecords` parameters to GetRecords request:

In [ ]:
#csw_records = {}
#sortby = SortBy([SortProperty('dc:title', 'ASC')])
#pagesize=10
#maxrecords=1000
#startposition = 0
#nextrecord = getattr(csw, 'results', 1)
#while nextrecord != 0:
#    csw.getrecords2(constraints=[anytext_query], startposition=startposition,
#                    maxrecords=pagesize, sortby=sortby)
#    csw_records.update(csw.records)
#    if csw.results['nextrecord'] == 0:
#        break
#    startposition += pagesize
#    if startposition >= maxrecords:
#        break
#csw.records.update(csw_records)
#records = '\n'.join(csw.records.keys())
#print('Found {} records.\n'.format(len(csw.records.keys())))
#for key, value in list(csw.records.items()):
#    print(f'identifier: {value.identifier}\ntype: {value.type}\ntitle: {value.title}\n')

The user then selects a record identifier and asks the catalogue to fetch the full record. Here we demonstrate how to obtain properties like title, bbox, full xml and links from the metadata record.

In [ ]:
#csw.getrecordbyid(id=['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'])
csw.getrecordbyid(id=[selected_record])

In [ ]:
#rec = csw.records['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE']
rec = csw.records[selected_record]

In [ ]:
rec.title

In [ ]:
rec.xml

In [ ]:
rec.references

In [ ]:
print("dataset bbox = (%s, %s, %s, %s)" % (rec.bbox.miny, rec.bbox.minx, rec.bbox.maxy, rec.bbox.maxx))

Using the [geolinks](https://github.com/geopython/geolinks) Python library we can filter the links that are of a specific type (here WMS and WCS links to be used for visualization)

In [ ]:
msg = 'geolink: {geolink}\nscheme: {scheme}\nURL: {url}\n'.format
for ref in rec.references:
    print(msg(geolink=sniff_link(ref['url']), **ref))

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WMS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WCS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

Finally we demonstrate how to show the record footprint on a map, using the [Folium](https://github.com/python-visualization/folium) Python library:

In [ ]:
centre_x = float(rec.bbox.minx) + ((float(rec.bbox.maxx) - float(rec.bbox.minx))/2)
centre_y = float(rec.bbox.miny) + ((float(rec.bbox.maxy) - float(rec.bbox.miny))/2)
m = folium.Map(location=[centre_y, centre_x], zoom_start=10, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(rec.bbox.miny), float(rec.bbox.minx)], [float(rec.bbox.maxy), float(rec.bbox.maxx)]]).add_to(m)
m

### System Catalogue Discovery through Federated Search

<span style="color:red;">**_THIS SECTION IS DISABLED, AS THE MUNDI CATALOGUE IS NO LONGER AVAILABLE_**<span>

This is an example of Federated Search using Mundi DIAS CSW

In [ ]:
# csw_url = 'https://resource-catalogue.mundi.eoepca.org/csw'
# # pycsw instance configured with 
# # federatedcatalogues=https://sentinel2.browse.catalog.mundiwebservices.com/csw

In [ ]:
# xml = b'''<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
# <csw:GetRecords xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.opengis.net/cat/csw/2.0.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd">
# 	<csw:DistributedSearch hopCount="2"/>
# 	<csw:Query typeNames="csw:Record">
# 		<csw:ElementSetName>brief</csw:ElementSetName>
# 	</csw:Query>
# </csw:GetRecords>
# '''

In [ ]:
# csw = CatalogueServiceWeb(csw_url)

In [ ]:
# csw.getrecords(xml=xml)

In [ ]:
# print(csw.results)

In [ ]:
# for record in csw.records:
#     print(csw.records[record].title)

### Workspace catalogue

In this part of the demo, the user will use the workspace resource catalogue endpoint to discover processing outputs and applications.  The workspace catalogue is local to the user, which also has federated access to the system catalogue.

User authenticates and the client receives an ID Token (JWT) that represents the user.<br>
For convenience within the Jupyter notebook we use a username/password authentication - but the primary mechanism is to rely upon external identity provision.<br>

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as user 'alice' and get ID Token
#-------------------------------------------------------------------------------
USER_ALICE="alice"
USER_ALICE_PASSWORD="changeme"
alice_access_token = get_user_token(USER_ALICE, USER_ALICE_PASSWORD)
headers = {
    'Authorization': 'Bearer ' + alice_access_token
}

We can retrieve the details of the user's workspace:

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{workspace_prefix}-{USER_ALICE}", headers=headers)
response.raise_for_status()

workspace_data = response.json()
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]

Alice uses the Processor Development Environment (PDE) to develop, test and package an application.
Alice's published outputs are:
* Docker image published to DockerHub
* Application Package (CWL) published to Resource Catalogue (TBD) and/or GitHub - accessible by href

We will first upload an application package to the user's workspace:

In [ ]:
import boto3
S3_ENDPOINT = "https://minio.develop.eoepca.org"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Now we register the application package:

In [ ]:
resource_url= f'{bucket_name}/application-package/s-expression/s-expression-0_0_2.cwl'
response = requests.post(
    f"{workspaces_endpoint}/{workspace_prefix}-{USER_ALICE}/register",
    headers=headers,
    json={
        "type": "application",
        "url": resource_url,
    }
)
response.raise_for_status()
response

In [ ]:
app_id = "s-expression"
app_version = "0_0_2"
application_name = f"{app_id}-{app_version}"

Setting up the authentication headers

In [ ]:
workspace_endpoint = f'https://resource-catalogue.{workspace_prefix}-{USER_ALICE}.{base_domain}/csw'

In [ ]:
csw = CatalogueServiceWeb(workspace_endpoint,timeout=30,headers=headers)

With a GetRecords request the user can see the number of total records in the workspace catalogue

In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
csw.records[app_id].references

In [ ]:
# Extract the URL for the Application Package
application_package_url = csw.records[app_id].references[1]['url']
#application_package_url = "https://app-packages.s3.fr-par.scw.cloud/nhi/app-nhi.dev.0.0.3.cwl"
print(f"Application Package URL = {application_package_url}")

Then, using the identifiers, the user can parse the links to the output data as well as any applications that are available ([Common Workflow Language [CWL]](https://www.commonwl.org))

In this part of the demo, Eric will use the workspace resource catalogue endpoint to discover processing output.  The workspace catalogue is local to the user, which also has federated access to the system catalogue.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as user 'bob' and get ID Token
#-------------------------------------------------------------------------------
USER_ERIC="eric"
USER_ERIC_PASSWORD="changeme"
eric_access_token = get_user_token(USER_ERIC, USER_ERIC_PASSWORD)

Setting up the authentication headers

In [ ]:
headers = {
    'Authorization': 'Bearer ' + eric_access_token
}

Eric uses his workspace catalogue to list all records identifier, type and title

In [ ]:
workspace_endpoint = f'https://resource-catalogue.{workspace_prefix}-{USER_ERIC}.{base_domain}/csw'
print(f"Eric's workspace (catalogue): {workspace_endpoint}")
csw = CatalogueServiceWeb(workspace_endpoint, timeout=30,headers=headers)
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')
csw.records[rec].references

### OpenSearch

In this part of the demo, the user will use the OpenSearch capability of the system resource catalogue to discover datasets.

In [ ]:
endpoint = system_catalogue_endpoint + '?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'

Here, the OWSLib OpenSearch client is used.

In [ ]:
os = OpenSearch(endpoint)

We use the description object to retrieve service metadata.

In [ ]:
os.description.shortname

In [ ]:
os.description.longname

In [ ]:
os.description.description

In [ ]:
os.description.urls

In [ ]:
os.description.tags

Then we perform a search using atom encoding.

In [ ]:
results = os.search('application/atom+xml')
len(results)

Another posibility is to use the HTTP GET requests directly with a generic client like requests.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
S = requests.Session()

Here, the user asks for the OpenSearch entrypoint template

In [ ]:
R = S.get(url=endpoint)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

A GetRecords request (in the context of OpenSearch) is performed

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

A collections search is also demonstrated, for Sentinel2 Level 2A results

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&eo:parentIdentifier=S2MSI2A'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can also use OpenSearch EO mathematical notation to filter based on other parameters

In [ ]:
# OpenSearch EO mathematical notation
# n1 equal to field = n1
# {n1,n2,…} equals to field=n1 OR field=n2 OR …
# [n1,n2] equal to n1 <= field <= n2
# [n1,n2[ equals to n1 <= field < n2
# ]n1,n2[ equals to n1 < field < n2
# ]n1,n2] equal to n1 < field  <= n2
# [n1 equals to n1<= field
# ]n1 equals to n1 < field
# n2] equals to field <= n2
# n2[ equals to field < n2

url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&eo:cloudCover=]20'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can get one record using the identifier through the OpenSearch EO API:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&recordids=S2B_MSIL1C_20190910T095029_N0208_R079_T33SUB_20190910T120214.SAFE'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can search using a bbox parameter:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&bbox=13.9,37,15.1,37.9'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

The user can search using a time parameter:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&time=2019-09-10/2019-09-12'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Or separate time start/stop parameters:

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&start=2019-09-10&stop=2019-09-12'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Finally, both system and workspace endpoints can be queried using a Federated Search query, where the user sends a request to the workspace catalogue which in turn sends a distributed search request to the system catalogue and aggregates the results for the user.

In [ ]:
url = system_catalogue_endpoint + '?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&typenames=csw:Record&elementsetname=full&resulttype=results&distributedSearch=TRUE&hopcount=1'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

Please note in the above result that the total of records is a combination of the system and workspace catalogues.

### OGC API Records demo

In this part of the demo, the user will use the system level resource catalogue OGC API Records endpoint to discover data datasets.
The `owslib.ogcapi.records` class of OWSLib is instantiated and service metadata are shown.

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'

In [ ]:
w = Records(system_catalogue_endpoint)

In [ ]:
w.url

Conformance classes supported by the OGC API Records server:

In [ ]:
w.conformance()

OpenAPI document of the OGC API Records server:

In [ ]:
w.api()

Collections available on the catalogue:

In [ ]:
w.collections()

In [ ]:
records = w.records()

In [ ]:
len(records)

The user can then specify the collection to search within the catalogue:

In [ ]:
my_catalogue = w.collection('metadata:main')

In [ ]:
my_catalogue['id']

Collection level queryables:

In [ ]:
w.collection_queryables('metadata:main')

Query the catalogue for all records:

In [ ]:
my_catalogue_query = w.collection_items('metadata:main')

In [ ]:
my_catalogue_query['numberMatched']

Metadata of first result:

In [ ]:
my_catalogue_query['features'][0]['properties'].keys()

In [ ]:
my_catalogue_query['features'][0]['properties']['title']

Query the catalogue using filters:

In [ ]:
#my_catalogue_query2 = w.collection_items('metadata:main', q='MSI')
my_catalogue_query2 = w.collection_items('metadata:main', bbox=[14.9,47.7,16.4,48.7])
# my_catalogue_query2 = w.collection_items('S2MSI2A', bbox=[14.9,47.7,16.4,48.7])

In [ ]:
my_catalogue_query2['numberMatched']

Full query result:

In [ ]:
[f['id'] for f in my_catalogue_query2['features']]

Text CQL query:

In [ ]:
my_catalogue_cql_text_query = w.collection_items('metadata:main', filter="title LIKE 'S2B_MSIL1C_%'")

In [ ]:
my_catalogue_cql_text_query['numberMatched']

In [ ]:
my_catalogue_cql_text_query['features'][0]['properties']['title']

JSON CQL query:

In [ ]:
my_catalogue_cql_json_query = w.collection_items('metadata:main', limit=1, cql={'op': '=', 'args': [{ 'property': 'title' }, 'S2B_MSIL1C_20190910T095029_N0208_R079_T33TXN_20190910T120910.SAFE']})

In [ ]:
my_catalogue_cql_json_query['features'][0]['properties']['title']

### Registration API demo

This is an example on how to use the EOEPCA Registration API to register metadata records in the Resource Catalogue.

In [ ]:
import requests
import datetime
import time
import json

In [ ]:
base_domain = "develop.eoepca.org"
registration_endpoint = f'https://registration-api-v1x.{base_domain}'

We will now register an ADES instance in the Registration API

In [ ]:
response = requests.post(
    f"{registration_endpoint}/register",
    json={
        "type": "ades",
        "url": "https://demo.pygeoapi.io/stable/processes",
    }
)
response.raise_for_status()
response

Check if the ADES is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{base_domain}/collections/metadata:main/items?type=service", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

We will now register a STAC Item from the Registration API JSON endpoint.

In [ ]:
url='https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/core-item.json'
stac_item=requests.get(url).text
#print(stac_item)

In [ ]:
response = requests.post(
    f"{registration_endpoint}/register-json",
    json=json.loads(stac_item)
)
response.raise_for_status()
response

Check if the STAC Item is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{base_domain}/collections/metadata:main/items/20201211_223832_CS2", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

### Registration API v2 demo

In [ ]:
import requests
import time
import json

In [ ]:
base_domain = "develop.eoepca.org"
registration_endpoint = f'https://registration-api.{base_domain}'

We will now register a STAC Item in the Registration API using OGC API - Processes

In [ ]:
response = requests.post(
    f"{registration_endpoint}/processes/register/execution",
    json={
        "inputs": {
            "type": "item",
            "source": "https://raw.githubusercontent.com/stac-utils/stac-examples/refs/heads/main/sentinel2/sentinel2-sample.json",
            "target": "https://resource-catalogue.develop.eoepca.org"
        }
    }
)
response.raise_for_status()
response

Check if the STAC Item is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{base_domain}/stac/collections/metadata:main/items/S2A_OPER_MSI_L2A_TL_SGS__20180524T190423_A015250_T26SKD_N02.08", headers=dict(f="json"))
response.raise_for_status()
response.json()

We will now unregister the STAC Item

In [ ]:
response = requests.post(
    f"{registration_endpoint}/processes/deregister/execution",
    json={
        "inputs": {
            "id": "S2A_OPER_MSI_L2A_TL_SGS__20180524T190423_A015250_T26SKD_N02.08",
            "target": "https://resource-catalogue.develop.eoepca.org"
        }
    }
)
response.raise_for_status()
response

Check if the STAC Item is actually unregistered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{base_domain}/stac/collections/metadata:main/items/S2A_OPER_MSI_L2A_TL_SGS__20180524T190423_A015250_T26SKD_N02.08", headers=dict(f="json"))
response.raise_for_status()
response.json()

We will now register a Record representing a workflow/process through Registration API

In [ ]:
json_str="""
{
  "inputs": {
    "source": {
      "id": "worldcereal_inference",
      "type": "Feature",
      "conformsTo": [
        "http://www.opengis.net/spec/ogcapi-records-1/1.0/req/record-core"
      ],
      "geometry": null,
      "properties": {
        "created": "2024-05-17T00:00:00Z",
        "updated": "2024-05-17T00:00:00Z",
        "type": "process",
        "title": "ESA worldcereal global maize detector",
        "description": "A maize detection algorithm.",
        "cost_estimate": 0.1,
        "cost_unit": "platform credits per km²",
        "keywords": [
          "agriculture",
          "crops",
          "apex_algorithm"
        ],
        "language": {
          "code": "en-US",
          "name": "English (United States)"
        },
        "languages": [
          {
            "code": "en-US",
            "name": "English (United States)"
          }
        ],
        "contacts": [
          {
            "name": "Kristof Van Tricht",
            "position": "Researcher",
            "organization": "VITO",
            "links": [
              {
                "href": "https://www.vito.be/",
                "rel": "about",
                "type": "text/html"
              },
              {
                "href": "https://github.com/kvantricht",
                "rel": "about",
                "type": "text/html"
              }
            ],
            "contactInstructions": "Contact via VITO",
            "roles": [
              "principal investigator"
            ]
          },
          {
            "name": "VITO",
            "links": [
              {
                "href": "https://www.vito.be/",
                "rel": "about",
                "type": "text/html"
              }
            ],
            "contactInstructions": "SEE WEBSITE",
            "roles": [
              "processor"
            ]
          }
        ],
        "themes": [
          {
            "concepts": [
              {
                "id": "AGRICULTURE"
              },
              {
                "id": "FIELD CROP PRODUCTS"
              }
            ],
            "scheme": "https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/sciencekeywords"
          }
        ],
        "formats": [
          {
            "name": "GeoTiff"
          }
        ],
        "license": "other"
      },
      "linkTemplates": [],
      "links": [
        {
          "rel": "openeo-process",
          "type": "application/json",
          "title": "openEO Process Definition",
          "href": "https://github.com/ESA-APEX/apex_algorithms/blob/main/openeo_udp/worldcereal_inference.json"
        },
        {
          "rel": "git",
          "type": "application/json",
          "title": "Git source repository",
          "href": "https://github.com/WorldCereal/worldcereal-classification.git"
        },
        {
          "rel": "service",
          "type": "application/json",
          "title": "CDSE openEO federation",
          "href": "https://openeofed.dataspace.copernicus.eu/"
        },
        {
          "rel": "license",
          "href": "https://apex.esa.int/license"
        }
      ]
    },
    "target": "https://resource-catalogue.develop.eoepca.org/",
    "type": "item"
  }
}
"""
response = requests.post(
    f"{registration_endpoint}/processes/register/execution",
    json=json.loads(json_str)
)
response.raise_for_status()
response

Check if the Record is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{base_domain}/stac/collections/metadata:main/items/worldcereal_inference", headers=dict(f="json"))
response.raise_for_status()
response.json()

We will now unregister the Record

In [ ]:
response = requests.post(
    f"{registration_endpoint}/processes/deregister/execution",
    json={
        "inputs": {
            "id": "worldcereal_inference",
            "target": "https://resource-catalogue.develop.eoepca.org"
        }
    }
)
response.raise_for_status()
response

Check if the Record is actually unregistered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{base_domain}/stac/collections/metadata:main/items/worldcereal_inference", headers=dict(f="json"))
response.raise_for_status()
response.json()

### OGC API Records Transactions demo

Examples of OGC API transactions using OGC API - Features - Part 4: Create, Replace, Update and Delete in the context of OGC API - Records metadata management.

In this part of the demo, the user will use the OWSLib client library to create/update/delete a record using the OGC API Transactions interface.

In [ ]:
import json
import requests
from owslib.ogcapi.records import Records

In [ ]:
record_data = '../data/sample-record.json'
base_domain = "develop.eoepca.org"
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'
collection_id = 'metadata:main'

In [ ]:
r = Records(system_catalogue_endpoint)

In [ ]:
cat = r.collection(collection_id)

In [ ]:
with open(record_data) as fh:
    data = json.load(fh)

In [ ]:
identifier = data['id']

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

In [ ]:
r.collection_item_delete(collection_id, identifier)

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Insert metadata.

In [ ]:
r.collection_item_create(collection_id, data)

# Similar approach using requests
#url = f'{system_catalogue_endpoint}/collections/metadata:main/items'
#headers = {'content-type': 'application/geo+json'}
#payload = open(record_data)
#req = requests.post(url, data=payload, headers=headers)

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Update metadata.

In [ ]:
data['properties']['description'] = "Update description"

In [ ]:
r.collection_item_update(collection_id, identifier, data)

Delete metadata.

In [ ]:
r.collection_item_delete(collection_id, identifier)

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Next, the user will ingest a STAC Item through the OGC API transactions.

In [ ]:
stac_data = '../data/S2B_MSIL2A_20190910T095029_N0213_R079_T33UWQ_20190910T124513.json'
with open(stac_data) as sf:
    si = json.load(sf)

In [ ]:
identifier = si['id']

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Delete metadata in case the identifier already exists.

In [ ]:
r.collection_item_delete(collection_id, identifier)

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Insert metadata.

In [ ]:
#Disabled due to POST issue https://github.com/geopython/pycsw/issues/809
#r.collection_item_create(collection_id, si)

url = f'{system_catalogue_endpoint}/collections/metadata:main/items'
headers = {'content-type': 'application/geo+json'}
payload = open(stac_data)
req = requests.post(url, data=payload, headers=headers)

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Delete metadata.

In [ ]:
r.collection_item_delete(collection_id, identifier)

In [ ]:
my_catalogue_query = r.collection_items(collection_id)
my_catalogue_query['numberMatched']

Alternative method to demo Transactions using curl from terminal:

In [ ]:
# insert metadata
# curl -v -H "Content-Type: application/geo+json" -XPOST https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items -d @sample-record.json
# update metadata
# curl -v -H "Content-Type: application/geo+json" -XPUT https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/foorecord -d @sample-record.json
# delete metadata
# curl -v -XDELETE https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/foorecord

### STAC API demo

In this part of the demo, the user will use the system level resource catalogue STAC API endpoint to discover data datasets.
The `pystac_client` library is used as a STAC client.

In [ ]:
from pystac_client import Client
from pystac_client import ConformanceClasses

In [ ]:
base_domain = "develop.eoepca.org"
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}/stac'

In [ ]:
catalog = Client.open(system_catalogue_endpoint)

STAC catalogue metadata:

In [ ]:
catalog.id

In [ ]:
catalog.title

In [ ]:
catalog.description

Conformance classes supported by the STAC API endpoint:

In [ ]:
dir(ConformanceClasses)

Validation of STAC API using `pystac_client`:

In [ ]:
catalog.conforms_to(ConformanceClasses.ITEM_SEARCH)

In [ ]:
catalog.conforms_to(ConformanceClasses.CORE)

Query the catalogue for all STAC items:

In [ ]:
mysearch = catalog.search(collections=['S2MSI2A'], max_items=10)
print(f"{mysearch.matched()} items found")

Query the STAC API using filters:

In [ ]:
mysearch = catalog.search(collections=['S2MSI2A'], bbox=[14.9,47.7,16.4,48.7], max_items=10)
print(f"{mysearch.matched()} items found")

Iterate through the query results:

In [ ]:
items = mysearch.items()
for item in items:
    print(item.id)

Show last STAC item JSON:

In [ ]:
# print(json.dumps(item.to_dict(), indent=2))
#print(item.to_dict())

In [ ]:
first_item = next(mysearch.items())
first_item

Make a Collection Search

In [ ]:
collection_search = catalog.collection_search(q="Sentinel")

Search results:

In [ ]:
collection_search.matched()

In [ ]:
collection_search.collection_list()

In [ ]:
for collection in collection_search.collections():
    print(collection.id)

### STAC API demo (eoAPI)

In [ ]:
from pystac_client import Client

In [ ]:
base_domain = "develop.eoepca.org"
stac_api_url = f'https://eoapi.{base_domain}/stac'

In [ ]:
# Search against the STAC API
catalog = Client.open(stac_api_url)
catalog.id

In [ ]:
# Select a collection
collections_filter = {
    "op": "=",
    "args": [{"property": "collection"}, "sentinel-2-l2a"],
}

In [ ]:
# Define your area of interest
AOI = {
  "type": "Polygon",
  "coordinates": [
    [
      [18.414015459767, 54.232763555655595],
      [20.04561585636452, 54.232763555655595],
      [20.04561585636452, 54.92536348887734],
      [18.414015459767, 54.92536348887734],
      [18.414015459767, 54.232763555655595]
    ]
  ]
}
spatial_filter = {"op": "s_intersects", "args": [{"property": "geometry"}, AOI]}

In [ ]:
# Define your time interval of interest
TIME_INTERVAL = ["2023-09-01T00:00:00Z", "2023-09-30T23:59:59Z"]
temporal_filter = {
    "op": "t_intersects",
    "args": [{"property": "datetime"}, {"interval": TIME_INTERVAL}],
}

In [ ]:
# Additional filters can be applied for other search criteria like <= maximum eo:cloud_cover in item properties
cloud_filter = {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 10]}

In [ ]:
# Define your search with CQL2 syntax
search = catalog.search(
    filter_lang="cql2-json",
    sortby=[{"direction": "asc", "field": "properties.datetime"}],
    filter={
      "op": "and",
      "args": [collections_filter, spatial_filter, temporal_filter, cloud_filter]
    }
)

In [ ]:
# Grab the first item from the search results and sign the assets
first_item = next(search.items())
first_item

### STAC Datacube demo

In this part of the demo, the user will register a STAC Item with datacube extension and use the system level resource catalogue to discover datacube datasets.

In [ ]:
import json
import requests
from owslib.ogcapi.records import Records

In [ ]:
record_data = '../data/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009-v2.0.7cds.json'
base_domain = "develop.eoepca.org"
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'
collection_id = 'metadata:main'

In [ ]:
r = Records(system_catalogue_endpoint)

In [ ]:
with open(record_data) as fh:
    data = json.load(fh)

In [ ]:
identifier = data['id']

In [ ]:
r.collection_item_delete(collection_id, identifier)

In [ ]:
r.collection_item_create(collection_id, data)

In [ ]:
my_catalogue_query = r.collection_items('metadata:main')
my_catalogue_query['numberMatched']

In [ ]:
my_catalogue_cql_text_query = r.collection_items('metadata:main', filter="anytext ILIKE '%datacube%'")
my_catalogue_cql_text_query['numberMatched']

In [ ]:
my_catalogue_cql_text_query['features'][0]['properties']['title']

In [ ]:
my_catalogue_cql_text_query['features'][0]

### QGIS catalogue demo

In this part of the demo, the user will use the system level resource catalogue endpoint to discover and visualize datasets through QGIS desktop application.

![QGIS main window with OSM loaded](img/Screenshot_QGIS_01.png)

The user starts up QGIS and loads some data, in this case the OSM base map. The MetaSearch tool is available on the toolbar

![MetaSearch main window](img/Screenshot_QGIS_02.png)

The user can add a new catalogue endpoint by pressing the New button, then needs to add the Resource Catalogue endpoint in the URL text box

![EOEPCA Resource Catalogue in QGIS](img/Screenshot_QGIS_03.png)

The service metadata for the Resource Catalogue are available from the Service Info button

![Service Info](img/Screenshot_QGIS_04.png)

The Service Capabilities are also available from the 'GetCapabilities Response' button

![Service Capabilities](img/Screenshot_QGIS_05.png)

The user moves to the 'Search' tab of MetaSearch main window to perform a catalogue search (in this case by adding a bounding box)

![Search with bbox](img/Screenshot_QGIS_06.png)

By pressing 'Search' the Resource Catalogue is performing a dataset search. Then by selecting a result, the dataset bbox is shown on the map.

![Search for datasets](img/Screenshot_QGIS_07.png)

The user can choose to view the selected dataset by selecting the 'Add Data' button, where MetaSearch automatically discovers available WMS, WFS, WCS links and enables the capability to load the data directly to QGIS map. In this case, the WMS and WCS endpoints are available from the Data Access links included in the catalogue record.

![Adding Data](img/Screenshot_QGIS_08.png)

By selecting 'Add WMS/WMTS' the default QGIS WMS dialog shows up. Here, the user can browse through the available layers offered by the Data Access component and select a layer to add to the map.

![QGIS WMS dialog](img/Screenshot_QGIS_09.png)

After selecting a layer, QGIS will add the layer to the map and user can preview the selected dataset

![Data preview](img/Screenshot_QGIS_10.png)

By double clicking the catalogue record, the user can also preview the record metadata

![Metadata preview](img/Screenshot_QGIS_11.png)